In [76]:
import billboard
import lyricsgenius
import pandas as pd
import spotipy
import sqlalchemy as db
from datetime import datetime
from psaw import PushshiftAPI
from spotipy.oauth2 import SpotifyClientCredentials

from api_keys import *

In [78]:
sp = spotipy.Spotify(client_credentials_manager = SpotifyClientCredentials(client_id = SPOTIFY_ID,
                                                                           client_secret = SPOTIFY_SECRET))

genius = lyricsgenius.Genius(GENIUS_ACCESS_TOKEN)
genius.verbose = False
genius.remove_section_headers = True

reddit = PushshiftAPI()

engine = db.create_engine(f"postgresql+psycopg2://{SQL_USER}:{SQL_PASS}@{SQL_HOST}/{SQL_DB}")
connection = engine.connect()

Our API wrappers:

In [3]:
def getSingleChart(date):
    
    def getSpotifyURI(song): # This needs to be worked a bit, sometimes the titles have guest artists' names
        try:
            return sp.search(q = f"{song.title} {song.artist}", type = "track", limit = 1)["tracks"]["items"][0]["uri"]
        except IndexError:
            return None
        
    chart = billboard.ChartData("hot-100", date = date)
    
    return pd.DataFrame( dict(song.__dict__, date = date, spotify_uri = getSpotifyURI(song)) for song in chart )


def getSingleSongFeatures(spotify_uri):
    features = sp.audio_features(spotify_uri)[0]
    
    if features is None:
        return {"uri": spotify_uri} # some kind of placeholder so we know it has no features
    
    return features


def getSingleSongMetadata(spotify_uri):
    song_info = sp.track(spotify_uri)
    song = genius.search_song(title = song_info["name"], artist = song_info["artists"][0]["name"])
    
    if song is None:
        return dict()
    
    return dict(song.to_dict(), spotify_uri = spotify_uri)

    
def redditSearch(search_function, subreddit, from_date, to_date):
    start_date = datetime(*[ int(k) for k in from_date.split('-') ])
    start_epoch = int(start_date.timestamp())
    
    end_date = datetime(*[ int(k) for k in to_date.split('-') ])
    end_epoch = int(end_date.timestamp())

    results = search_function(after = start_epoch, before = end_epoch, subreddit = subreddit)
    
    return pd.DataFrame( row.d_ for row in results )

Aggregating those calls:

In [4]:
def getCharts(dates):
    return pd.concat( getSingleChart(date) for date in dates )

def getSongFeatures(spotify_uris):
    return pd.DataFrame( getSingleSongFeatures(uri) for uri in spotify_uris if uri is not None )

def getSongMetadata(spotify_uris):
    return pd.DataFrame( getSingleSongMetadata(uri) for uri in spotify_uris if uri is not None )

def getPosts(subreddit, from_date, to_date):
    return redditSearch(reddit.search_submissions, subreddit, from_date, to_date)

def getComments(subreddit, from_date, to_date):
    return redditSearch(reddit.search_comments, subreddit, from_date, to_date)

Fetching the data:

In [99]:
months = [ f"{y}-{m:02d}-01" for y in range(2015, 2019+1) for m in range(1, 12+1) ]

print(months[0], "to", months[-1])

2015-01-01 to 2019-12-01


In [5]:
chartsTable = getCharts(months)

In [32]:
print(chartsTable.columns.values)

['title' 'artist' 'image' 'peakPos' 'lastPos' 'weeks' 'rank' 'isNew'
 'date' 'spotify_uri']


In [37]:
chartsTable

,title,artist,image,peakPos,lastPos,weeks,rank,isNew,date,spotify_uri
0,The Box,Roddy Ricch,None,1,1,7,1,False,2020-02-01,spotify:track:0nbXyq5TXYPCO7pr3N8S4I
1,Life Is Good,Future Featuring Drake,None,2,2,2,2,False,2020-02-01,None
2,Godzilla,Eminem Featuring Juice WRLD,None,3,0,1,3,True,2020-02-01,None
3,Circles,Post Malone,None,1,3,21,4,False,2020-02-01,spotify:track:21jGcNKet2qwijlDFuPiPb
4,Memories,Maroon 5,None,2,4,18,5,False,2020-02-01,spotify:track:2b8fOow8UzyDFAE27YhOZM
...,...,...,...,...,...,...,...,...,...,...
95,Ridin' Roads,Dustin Lynch,None,47,85,17,96,False,2020-03-01,spotify:track:2VPmBOuy7ZAOFSzKwW2IEt
96,Me And My Guitar,A Boogie Wit da Hoodie,None,58,58,2,97,False,2020-03-01,spotify:track:6cZH4rX1KTt1aJ3Ql6Ynja
97,Feel Me,Selena Gomez,None,98,0,1,98,True,2020-03-01,spotify:track:6XXYdF6pJR1K3wKvuxmu7n
98,Vete,Bad Bunny,None,33,96,13,99,False,2020-03-01,spotify:track:5DxXgozhkPLgrbKFY91w0c


In [ ]:
chartsTable.to_sql("charts", connection, if_exists = "replace") # creates a new table

del chartsTable

In [7]:
audioTable = getSongFeatures(set(chartsTable.spotify_uri)) # take the set in case songs are on the charts for many months

retrying ...3secs


In [33]:
print(audioTable.columns.values)

['danceability' 'energy' 'key' 'loudness' 'mode' 'speechiness'
 'acousticness' 'instrumentalness' 'liveness' 'valence' 'tempo' 'type'
 'id' 'uri' 'track_href' 'analysis_url' 'duration_ms' 'time_signature']


In [38]:
audioTable

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.717,0.5980,5,-5.613,0,0.0809,0.2200,0.000000,0.1220,0.496,153.905,audio_features,4SDgTLDYrJ2UrHbkRkg7MD,spotify:track:4SDgTLDYrJ2UrHbkRkg7MD,https://api.spotify.com/v1/tracks/4SDgTLDYrJ2U...,https://api.spotify.com/v1/audio-analysis/4SDg...,151040,4
1,0.677,0.3300,2,-11.198,1,0.1490,0.4740,0.000871,0.0640,0.336,68.985,audio_features,2Yv2mHzr5AQavVdwQjEokV,spotify:track:2Yv2mHzr5AQavVdwQjEokV,https://api.spotify.com/v1/tracks/2Yv2mHzr5AQa...,https://api.spotify.com/v1/audio-analysis/2Yv2...,298440,4
2,0.682,0.8090,3,-4.331,0,0.3780,0.4910,0.000000,0.2170,0.812,180.088,audio_features,2OUY5NdALupSoIQN9ZBH1a,spotify:track:2OUY5NdALupSoIQN9ZBH1a,https://api.spotify.com/v1/tracks/2OUY5NdALupS...,https://api.spotify.com/v1/audio-analysis/2OUY...,138683,4
3,0.710,0.8080,2,-6.703,0,0.0433,0.4400,0.000773,0.0734,0.392,99.989,audio_features,6XXYdF6pJR1K3wKvuxmu7n,spotify:track:6XXYdF6pJR1K3wKvuxmu7n,https://api.spotify.com/v1/tracks/6XXYdF6pJR1K...,https://api.spotify.com/v1/audio-analysis/6XXY...,226003,4
4,0.591,0.5850,2,-6.350,1,0.0277,0.1430,0.000000,0.1090,0.324,110.940,audio_features,7lidXGPXPYLNThITAOTlkK,spotify:track:7lidXGPXPYLNThITAOTlkK,https://api.spotify.com/v1/tracks/7lidXGPXPYLN...,https://api.spotify.com/v1/audio-analysis/7lid...,205473,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,0.768,0.5650,4,-7.203,0,0.0525,0.1430,0.000000,0.2770,0.307,129.866,audio_features,4RhRDVXc3xFojyyjyh1jmN,spotify:track:4RhRDVXc3xFojyyjyh1jmN,https://api.spotify.com/v1/tracks/4RhRDVXc3xFo...,https://api.spotify.com/v1/audio-analysis/4RhR...,175527,4
99,0.794,0.7930,11,-4.521,0,0.0842,0.0125,0.000000,0.0952,0.677,123.941,audio_features,6WrI0LAC5M1Rw2MnX2ZvEg,spotify:track:6WrI0LAC5M1Rw2MnX2ZvEg,https://api.spotify.com/v1/tracks/6WrI0LAC5M1R...,https://api.spotify.com/v1/audio-analysis/6WrI...,183290,4
100,0.610,0.0316,9,-15.186,1,0.0486,0.8960,0.004610,0.1080,0.370,151.964,audio_features,4jXl6VtkFFKIt3ycUQc5LT,spotify:track:4jXl6VtkFFKIt3ycUQc5LT,https://api.spotify.com/v1/tracks/4jXl6VtkFFKI...,https://api.spotify.com/v1/audio-analysis/4jXl...,170360,4
101,0.733,0.3890,10,-8.468,0,0.0738,0.0513,0.000000,0.1410,0.244,157.878,audio_features,6fWa5CRgy0z30OeGZyMBvD,spotify:track:6fWa5CRgy0z30OeGZyMBvD,https://api.spotify.com/v1/tracks/6fWa5CRgy0z3...,https://api.spotify.com/v1/audio-analysis/6fWa...,159670,3


In [ ]:
audioTable.to_sql("audio", connection, if_exists = "replace")

del audioTable

In [9]:
lyricsTable = getSongMetadata(set(chartsTable.spotify_uri)) # slow

In [34]:
print(lyricsTable.columns.values)

['title' 'album' 'year' 'lyrics' 'image' 'spotify_uri']


In [39]:
lyricsTable

,title,album,year,lyrics,image,spotify_uri
0,Panini (DaBaby Remix),None,2019-09-13,"D-D-Daytrip took it to ten (Hey)\n\nAyy, Panin...",https://images.genius.com/3e6ec8ff347a1d2e577c...,spotify:track:4SDgTLDYrJ2UrHbkRkg7MD
1,Hand Me Downs,Circles,2020-01-17,I'm thinkin' maybe I should thank you\nOh baby...,https://images.genius.com/f5528ff2f79b8c9aaf79...,spotify:track:2Yv2mHzr5AQavVdwQjEokV
2,Bad Bad,"Still Flexin, Still Steppin",2020-02-17,"I got Hitman on the beat\nAKel (Yes, sir)\n\nY...",https://images.genius.com/60123290eae91ff1bde9...,spotify:track:2OUY5NdALupSoIQN9ZBH1a
3,Feel Me,Rare (Deluxe Edition),2020-02-21,"No one love you like I love ya\nNever cheat, n...",https://images.genius.com/ee8e77b6590309a95e66...,spotify:track:6XXYdF6pJR1K3wKvuxmu7n
4,You should be sad,Manic (Target Exclusive),2020-01-10,I wanna start this out and say\nI gotta get it...,https://images.genius.com/8741a6cd125fe3827050...,spotify:track:7lidXGPXPYLNThITAOTlkK
...,...,...,...,...,...,...
98,Long RD,"Still Flexin, Still Steppin",2020-02-21,"(Yung Lan on the track)\n(Felipe S)\nYeah-eah,...",https://images.genius.com/60123290eae91ff1bde9...,spotify:track:4RhRDVXc3xFojyyjyh1jmN
99,Don’t Start Now,Future Nostalgia,2019-11-01,"If you don't wanna see me\n\nDid a full 180, c...",https://images.genius.com/8d61ec5fd527a5e54249...,spotify:track:6WrI0LAC5M1Rw2MnX2ZvEg
100,Circles,Circles,2020-01-17,"Well, this is what it look like right before y...",https://images.genius.com/f5528ff2f79b8c9aaf79...,spotify:track:4jXl6VtkFFKIt3ycUQc5LT
101,Heart on Ice,PTSD,2019-05-22,"SpeakerBangerz\nLook, uh, look\n\nHeart been b...",https://images.genius.com/249cb2345d6f6face185...,spotify:track:6fWa5CRgy0z30OeGZyMBvD


In [ ]:
lyricsTable.to_sql("lyrics", connection, if_exists = "replace")

del lyricsTable

In [11]:
postsTable = getPosts("news", "2020-02-01", "2020-02-02")

In [35]:
print(postsTable.columns.values)

['all_awardings' 'allow_live_comments' 'author' 'author_flair_css_class'
 'author_flair_richtext' 'author_flair_text' 'author_flair_type'
 'author_fullname' 'author_patreon_flair' 'author_premium' 'awarders'
 'can_mod_post' 'contest_mode' 'created_utc' 'domain' 'full_link'
 'gildings' 'id' 'is_crosspostable' 'is_meta' 'is_original_content'
 'is_reddit_media_domain' 'is_robot_indexable' 'is_self' 'is_video'
 'link_flair_background_color' 'link_flair_richtext'
 'link_flair_text_color' 'link_flair_type' 'locked' 'media_only'
 'no_follow' 'num_comments' 'num_crossposts' 'over_18'
 'parent_whitelist_status' 'permalink' 'pinned' 'pwls'
 'removed_by_category' 'retrieved_on' 'score' 'selftext' 'send_replies'
 'spoiler' 'stickied' 'subreddit' 'subreddit_id' 'subreddit_subscribers'
 'subreddit_type' 'thumbnail' 'title' 'total_awards_received' 'url'
 'whitelist_status' 'wls' 'created' 'post_hint' 'preview'
 'thumbnail_height' 'thumbnail_width' 'media' 'media_embed' 'secure_media'
 'secure_media_e

In [40]:
postsTable

,all_awardings,allow_live_comments,author,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_type,author_fullname,author_patreon_flair,author_premium,...,created,post_hint,preview,thumbnail_height,thumbnail_width,media,media_embed,secure_media,secure_media_embed,link_flair_text
0,[],False,121kiwi,None,[],None,text,t2_3z42zcfk,False,False,...,1.580656e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],False,121kiwi,None,[],None,text,t2_3z42zcfk,False,False,...,1.580656e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],False,121kiwi,None,[],None,text,t2_3z42zcfk,False,False,...,1.580656e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],False,121kiwi,None,[],None,text,t2_3z42zcfk,False,False,...,1.580656e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[],False,121kiwi,None,[],None,text,t2_3z42zcfk,False,False,...,1.580656e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1580,[],False,RollinonEase,None,[],None,text,t2_15x2g9zz,False,False,...,1.580570e+09,link,"{'enabled': False, 'images': [{'id': 'dBZGIXYz...",84.0,140.0,NaN,NaN,NaN,NaN,NaN
1581,[],False,sanj12345,None,[],None,text,t2_4kmfk8ta,False,False,...,1.580570e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1582,[],False,Sinnivar,None,[],None,text,t2_154lpxwa,False,False,...,1.580569e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1583,[],False,fatouakinajma,None,[],None,text,t2_3zxw1nd7,False,False,...,1.580569e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
postsTable.astype(str).to_sql("posts", connection, if_exists = "replace") # cast to string to insert dict objects

del postsTable

In [85]:
commentsTable = getComments("news", "2020-02-01", "2020-02-02")

In [36]:
print(commentsTable.columns.values)

['all_awardings' 'associated_award' 'author'
 'author_flair_background_color' 'author_flair_css_class'
 'author_flair_richtext' 'author_flair_template_id' 'author_flair_text'
 'author_flair_text_color' 'author_flair_type' 'author_fullname'
 'author_patreon_flair' 'author_premium' 'awarders' 'body'
 'collapsed_because_crowd_control' 'created_utc' 'gildings' 'id'
 'is_submitter' 'link_id' 'locked' 'no_follow' 'parent_id' 'permalink'
 'retrieved_on' 'score' 'send_replies' 'stickied' 'subreddit'
 'subreddit_id' 'total_awards_received' 'created' 'distinguished'
 'author_cakeday']


In [41]:
commentsTable

,all_awardings,associated_award,author,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,author_flair_text_color,author_flair_type,...,retrieved_on,score,send_replies,stickied,subreddit,subreddit_id,total_awards_received,created,distinguished,author_cakeday
0,[],None,r_r_36,None,None,[],None,None,None,text,...,1580630409,1,True,False,news,t5_2qh3l,0,1.580656e+09,NaN,NaN
1,[],None,up766570,None,None,[],None,None,None,text,...,1580630409,1,True,False,news,t5_2qh3l,0,1.580656e+09,NaN,NaN
2,[],None,ViolableOlive,None,None,[],None,None,None,text,...,1580630400,1,True,False,news,t5_2qh3l,0,1.580656e+09,NaN,NaN
3,[],None,r_r_36,None,None,[],None,None,None,text,...,1580630381,1,True,False,news,t5_2qh3l,0,1.580656e+09,NaN,NaN
4,[],None,[deleted],,None,NaN,None,None,dark,NaN,...,1580630378,1,True,False,news,t5_2qh3l,0,1.580656e+09,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21396,[],None,gameofthrombosis,None,None,[],None,None,None,text,...,1580544048,1,True,False,news,t5_2qh3l,0,1.580569e+09,NaN,NaN
21397,[],None,not_usually_serious,None,None,[],None,None,None,text,...,1580544041,1,True,False,news,t5_2qh3l,0,1.580569e+09,NaN,NaN
21398,[],None,SnackyDoo,None,None,[],None,None,None,text,...,1580544024,1,True,False,news,t5_2qh3l,0,1.580569e+09,NaN,NaN
21399,[],None,[deleted],,None,NaN,None,None,dark,NaN,...,1580544021,1,True,False,news,t5_2qh3l,0,1.580569e+09,NaN,NaN


In [79]:
commentsTable.astype(str).to_sql("comments", connection, if_exists = "replace")

del commentsTable

In [95]:
connection.close()

AttributeError: module 'sqlalchemy' has no attribute 'dispose'